In [2]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from google.colab import drive

In [1]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = pd.read_csv('/content/drive/MyDrive/data_grants_114_v20240218.csv', sep=";")
data.head(10)

<ipython-input-4-6443e50f87a7>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/data_grants_114_v20240218.csv', sep=";")


,name,contest,year,direction,date_req,region,org,inn,ogrn,implem_start,...,goal,tasks,soc_signif,pj_geo,target_groups,contacts,req_num,link,okato,oktmo
0,Воскрешая имена Возрождаем людей,Первый конкурс 2017,2017,Сохранение исторической памяти,2017-04-19,Волгоградская область,Волгоградская региональная поисковая обществен...,3446997050,1.113400e+12,2017-09-01,...,Цель: Увековечение памяти защитников Отечества...,Организация и проведении поисковых работ в Вол...,Проведение молодежных обучающих поисковых Вахт...,Волгоградская область,"Ветераны, Многодетные семьи, Молодежь и студен...","400074, г Волгоград, ул им Огарева, д 18, кв 6...",17-1-002780,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,18000000,18000000
1,Организация детской секции водно-моторного спо...,Первый конкурс 2017,2017,"Охрана здоровья граждан, пропаганда здорового ...",2017-04-19,Красноярский край,Местная общественная организация г. Красноярск...,2466063169,1.022400e+12,2018-02-25,...,Организация бесплатных занятий по водно-моторн...,"Познакомить детей с водно-моторным спортом, на...",На сегодняшний день развитие водно-моторного с...,г. Красноярск,"Дети и подростки, Многодетные семьи, Дети-сиро...","660093, г Красноярск, остров Молокова, д 23 ст...",17-1-001989,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,4000000,4000000
2,Благотворительные гастроли инклюзивного аудио-...,Первый конкурс 2017,2017,Поддержка проектов в области культуры и искусства,2017-04-19,Тюменская область,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ «МОЛОДЕЖ...,7203209434,1.097200e+12,2017-09-01,...,"Приобщение слабовидящих и слепых людей, прожив...","Разработка, согласование и утверждение гастрол...",В РФ около 278000 слабовидящих и 110000 полнос...,Тюменская область.,Люди с ограниченными возможностями здоровья,"625007, Тюменская обл, г Тюмень, ул Валерии Гн...",17-1-002371,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,71000000,71000000
3,"Мы сильнее,когда помогаем.",Первый конкурс 2017,2017,"Охрана здоровья граждан, пропаганда здорового ...",2017-04-19,Воронежская область,Воронежская областная спортивная общественная ...,3662989825,1.133600e+12,2017-09-08,...,Основная идея нашего проекта (создание социаль...,"1.Подготовить песню (Гимн) \Мы сильнее, когда ...",Мы рукоплещем успехам наших спортсменов - пара...,Проект планируется реализовывать на территории...,Граждане России.,"394020, г Воронеж, ул 9 Января, д 264, кв 37 В...",17-1-000413,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,20000000,20000000
4,"Интерактивный семейный спектакль, адаптированн...",Первый конкурс 2017,2017,"Поддержка семьи, материнства, отцовства и детства",2017-04-20,Тюменская область,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ «МОЛОДЕЖ...,7203209434,1.097200e+12,2017-10-01,...,Интеграция семей с глухими детьми в общедоступ...,Создание экспериментального семейного инклюзив...,По данным ВОЗ в мире насчитывается 490 млн. сл...,Тюменская область,Люди с ограниченными возможностями здоровья,"625007, Тюменская обл, г Тюмень, ул Валерии Гн...",17-1-002987,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,71000000,71000000
5,Просветительский онлайн-проект “Живые голоса Д...,Первый конкурс 2017,2017,Сохранение исторической памяти,2017-04-21,Москва,некоммерческое партнерство историко-просветите...,7734271230,1.147799e+12,2018-01-01,...,Через публикацию в доступной форме исторически...,Отобрать и представить аудитории наиболее ярки...,Социальная актуальность проекта обусловлена на...,город Москва,"Дети и подростки, Молодежь и студенты","123098, г Москва, ул Маршала Василевского, д 9...",17-1-002896,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,45000000,45000000
6,Противодействие радикализму и экстремизму на т...,Первый конкурс 2017,2017,Укрепление межнационального и межрелигиозного ...,2017-04-21,Свердловская область,ЦЕНТРАЛИЗОВАННАЯ РЕЛИГИОЗНАЯ ОРГАНИЗАЦИЯ ДУХОВ...,6658994835,1.136600e+12,2017-09-01,...,Организация продуктивного противодействия возн...,В рамках проекта планируется провести духовную...,В российском обществ

In [5]:
data = data.sample(10000)
data.index = range(0,10000)

In [6]:

y = data["winner"]
columns_to_drop = ['link','ogrn', 'okato', 'oktmo', 'req_num', 'implem_end',
                   'contacts', 'year','rate', 'date_req', 'org', 'pj_geo', 'winner', 'cofunding','target_groups', 'tasks',  'contest', 'money_req_grant']
x = data.drop(columns = columns_to_drop)



X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)
vc1 = TfidfVectorizer(max_features=4000)
description_text = vc1.fit_transform(X_train['description'])
test_description_text = vc1.transform(X_test['description'])

vc2 = TfidfVectorizer(max_features=4000)
soc_signif = vc2.fit_transform(X_train['soc_signif'])
test_soc_signif = vc2.transform(X_test['soc_signif'])

vc3 = TfidfVectorizer(max_features=4000)
region = vc3.fit_transform(X_train['region'])
test_region = vc3.transform(X_test['region'])

vc4 = TfidfVectorizer(max_features=4000)
name = vc4.fit_transform(X_train['name'])
test_name = vc4.transform(X_test['name'])

vc5 = TfidfVectorizer(max_features=4000)
implem_start = vc5.fit_transform(X_train['implem_start'])
test_implem_start = vc5.transform(X_test['implem_start'])





features_description = [f'text_5{i}' for i in range(description_text.shape[1])]
test_features_description = [f'text_5{i}' for i in range(test_description_text.shape[1])]

features_soc_signif = [f'text_8{i}' for i in range(soc_signif.shape[1])]
test_features_soc_signif = [f'text_8{i}' for i in range(test_soc_signif.shape[1])]

features_region = [f'text_9{i}' for i in range(region.shape[1])]
test_features_region = [f'text_9{i}' for i in range(test_region.shape[1])]

features_name = [f'text_10{i}' for i in range(name.shape[1])]
test_features_name = [f'text_10{i}' for i in range(test_name.shape[1])]

features_implem_start = [f'text_11{i}' for i in range(implem_start.shape[1])]
test_features_implem_start = [f'text_11{i}' for i in range(test_implem_start.shape[1])]




description_text = pd.DataFrame(description_text.toarray(), columns = features_description)
test_description_text = pd.DataFrame(test_description_text.toarray(), columns = test_features_description)

soc_signif = pd.DataFrame(soc_signif.toarray(), columns = features_soc_signif)
test_soc_signif = pd.DataFrame(test_soc_signif.toarray(), columns = test_features_soc_signif)

region = pd.DataFrame(region.toarray(), columns = features_region)
test_region = pd.DataFrame(test_region.toarray(), columns = test_features_region)

name = pd.DataFrame(name.toarray(), columns = features_name)
test_name = pd.DataFrame(test_name.toarray(), columns = test_features_name)

implem_start = pd.DataFrame(implem_start.toarray(), columns = features_implem_start)
test_implem_start = pd.DataFrame(test_implem_start.toarray(), columns = test_features_implem_start)


X_train = pd.concat([soc_signif, description_text, region], axis='columns')

X_test = pd.concat([test_soc_signif, test_description_text, test_region], axis='columns')

In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компилируйте модель
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Обучите модель
model.fit(X_train, y_train, epochs=10, batch_size=32)



Epoch 1/10
250/250 [==============================] - 4s 13ms/step - loss: 0.5164 - accuracy: 0.7906
Epoch 2/10
238/250 [===========================>..] - ETA: 0s - loss: 0.4222 - accuracy: 0.8066

KeyboardInterrupt: 